In [63]:
import opentrons_json as oj
from opentrons_json import plates,commands
import uuid
from jsonschema import validate
import json
with open('opentrons_json/protocol-schema.json') as f:
    data = json.load(f)
import pandas as pd

In [64]:
pipette = str(uuid.uuid4())
tip_box = "tip_box"
trash = "trash"

protocol = {
    "protocol-schema": "1.0.0",

  "metadata": {
    "protocol-name": "Example transfer protocol",
    "author": "Keoni Gandall",
  },
    "default-values": oj.default_values,
    "designer-application": oj.designer_application,
    "robot": oj.ot2_standard,
    "pipettes": {pipette:oj.pipette("left", "p300_single", "p300_single_v1.3")},
    "labware": {
        tip_box: oj.labware('10',"opentrons-tiprack-300ul",name="300ul tip box"),
        trash: oj.labware('12',"fixed-trash",name="Trash yo")
    },
    "procedure": [
        {"subprocedure": [
        oj.commands.aspirate(pipette,50,plate_1,'A1'),
        oj.commands.dispense(pipette,50,plate_2,'A2')]}]
}

In [70]:
options = ['glycerol-plate_1',
 'glycerol-plate_10',
 'glycerol-plate_11',
 'glycerol-plate_12',
 'glycerol-plate_13',
 'glycerol-plate_15',
 'glycerol-plate_16',
 'glycerol-plate_17',
 'glycerol-plate_18',
 'glycerol-plate_19',
 'glycerol-plate_2',
 'glycerol-plate_20',
 'glycerol-plate_21',
 'glycerol-plate_22',
 'glycerol-plate_23',
 'glycerol-plate_24',
 'glycerol-plate_3',
 'glycerol-plate_4',
 'glycerol-plate_5',
 'glycerol-plate_6',
 'glycerol-plate_8']

collections_loc = pd.read_csv('./../../analysis/collections_loc.csv')[['gene_id','organism','plate_location','address','target','target_plate']]

for option in options:
    desired_plate = collections_loc[collections_loc['plate_location'] == option]
    list_of_target_plates = list(set(desired_plate['target_plate'].tolist()))
    
    new_protocol=json.loads(json.dumps(protocol)) # set new protocol instance
    commands = [] # set new commands
    
    new_protocol['labware'][option] = oj.labware('11','96-flat',name=option)
    
    for plate_index,target_plate in enumerate(list_of_target_plates):
        new_protocol['labware'][target_plate] = oj.labware(str(plate_index+1),'96-flat',name=target_plate)
        desired_scope = desired_plate[desired_plate['target_plate'] == target_plate].reset_index(drop=True)
        
        for index,row in desired_scope.iterrows():
            commands.append(oj.commands.pick_up_tip(pipette,tip_box,plates.standard96.locations[index]))
            commands.append(oj.commands.aspirate(pipette,50,option,row['address']))
            commands.append(oj.commands.dispense(pipette,50,target_plate,plates.standard96.locations[int(row['target'])]))
            commands.append(oj.commands.drop_tip(pipette,trash,'A1'))
    
    new_protocol['procedure'] = [{"subprocedure": commands}]
    validate(instance=new_protocol,schema=data)
    with open('/home/koeng/gits/opentrons_json/freegenes_move/{}.json'.format(option), 'w') as fp:
        json.dump(new_protocol, fp)
        
    

In [59]:
print(new_protocol)

{'protocol-schema': '1.0.0', 'metadata': {'protocol-name': 'Example transfer protocol', 'author': 'Keoni Gandall'}, 'default-values': {'aspirate-flow-rate': {'p10_single': 5, 'p10_multi': 5, 'p50_single': 25, 'p50_multi': 25, 'p300_single': 150, 'p300_multi': 150, 'p1000_single': 500}, 'dispense-flow-rate': {'p10_single': 10, 'p10_multi': 10, 'p50_single': 50, 'p50_multi': 50, 'p300_single': 300, 'p300_multi': 300, 'p1000_single': 1000}, 'aspirate-mm-from-bottom': 1, 'dispense-mm-from-bottom': 0.5, 'touch-tip-mm-from-top': -1}, 'designer-application': {'application-name': 'opentrons_json', 'application-version': '1.0.0', 'data': {}}, 'robot': {'model': 'OT-2 Standard'}, 'pipettes': {'9dfd597f-9e1b-4735-a104-66f6c19b0a82': {'type': 'pipette', 'mount': 'left', 'name': 'p300_single', 'model': 'p300_single_v1.3'}}, 'labware': {'tip_box': {'slot': '10', 'model': 'opentrons-tiprack-300ul', 'name': None}, 'glycerol-plate_8': {'slot': '11', 'model': '96-flat', 'name': None}, 'Mycoplasma_syn3_2